# 한국경제 기사 크롤링

In [ ]:
import requests, pandas as pd, time
from newspaper import Article
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

BASE_URL = "https://www.hankyung.com/article/"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/127.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    "Referer": "https://www.hankyung.com/",
}

def crawl_url(url):
    # 단일 URL 크롤링: requests로 가져와 newspaper 파싱 (403시 /amp 폴백)
    try:
        r = requests.get(url, headers=HEADERS, timeout=12)
        if r.status_code == 403:
            r = requests.get(url.rstrip("/") + "/amp", headers=HEADERS, timeout=12)
        if r.status_code in (403, 404):
            return None
        r.raise_for_status()

        art = Article(url, language='ko')
        art.set_html(r.text)
        art.parse()
        if not art.title or not art.text:
            return None
        return {"url": url, "title": art.title, "text": art.text, "publish_date": art.publish_date}
    except:
        return None

def crawl_i_articles(date_str, max_workers=27):
    """i 기사 크롤링 (0000~9999)"""
    urls = [f"{BASE_URL}{date_str}{num}" for num in range(85000, 85700)]
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = {ex.submit(crawl_url, u): u for u in urls}
        for fut in tqdm(as_completed(futures), total=len(futures), desc=f"I {date_str}"):
            res = fut.result()
            if res:
                results.append(res)
            time.sleep(0.06)  # 너무 빠른 요청 방지(차단 완화)
    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    # df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/hankyung_i_{date_str}.csv", index=False)
    return df

# 사용 예시
if __name__ == "__main__":
    day_str = "20250819"
    out_df = crawl_i_articles(day_str)
    out_df.head()

I 20250819: 100%|██████████| 700/700 [00:45<00:00, 15.47it/s]


                                              url  \
0  https://www.hankyung.com/article/2025081985166   
1  https://www.hankyung.com/article/2025081985194   
2  https://www.hankyung.com/article/2025081985184   
3  https://www.hankyung.com/article/2025081985214   
4  https://www.hankyung.com/article/2025081985254   

                           title  \
0  금투협, K-OTC서 상폐기업 장외거래 6개월간 지원   
1                           한국경제   
2                           한국경제   
3                           한국경제   
4                           한국경제   

                                                text              publish_date  
0  K-OTC에 상장폐지지정기업부 신설…내년부터 운영\n\n금융투자협회 자료사진. 사진... 2025-08-19 14:37:05+09:00  
1  트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...                       NaT  
2  트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...                       NaT  
3  트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...                       NaT  
4  트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...       

In [22]:
out_df

,url,title,text,publish_date
0,https://www.hankyung.com/article/2025081985166,"금투협, K-OTC서 상폐기업 장외거래 6개월간 지원",K-OTC에 상장폐지지정기업부 신설…내년부터 운영\n\n금융투자협회 자료사진. 사진...,2025-08-19 14:37:05+09:00
1,https://www.hankyung.com/article/2025081985194,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
2,https://www.hankyung.com/article/2025081985184,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
3,https://www.hankyung.com/article/2025081985214,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
4,https://www.hankyung.com/article/2025081985254,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
5,https://www.hankyung.com/article/2025081985264,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
6,https://www.hankyung.com/article/2025081985394,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
7,https://www.hankyung.com/article/2025081985494,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
8,https://www.hankyung.com/article/2025081985544,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
9,https://www.hankyung.com/article/2025081985595,"한미반도체, 마이크론과 동맹…매출 1조 노린다",영상 모듈 닫기\n\n앵커>고대역폭메모리(HBM) 패권을 둘러싼 글로벌 메모리 반도...,2025-08-19 14:26:02+09:00
